In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl

In [8]:
#import seaborn as sns

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [10]:
run ingredient_preprocessing.py

Loading data...done!
Cleaning ingredients in training data...done!
Cleaning ingredients in testing data...done!
Saving processed ingredients to csv...done!


In [11]:
train_data = pd.read_json("train_cleaned_v1.json")
test_data  = pd.read_json("test_cleaned_v1.json")

In [12]:
train_data.head()

,id,cuisine,ingredients
0,0,spanish,"[mussels, pepper, garlic, saffron threads, oli..."
1,1,mexican,"[tomatoes, red onions, paprika, salt, corn tor..."
2,2,french,"[chicken broth, truffles, pimentos, green pepp..."
3,3,chinese,"[ginger, sesame oil, peas, cooked rice, bell p..."
4,4,italian,"[orange peel, cookies, vanilla ice cream, gran..."


In [13]:
test_data.head()

,id,ingredients
0,5,"[mushrooms, onion, tomato sauce, cheese, orega..."
1,7,"[garlic, brown rice, sour cream, chicken, red ..."
2,11,"[lime juice, sesame oil, garlic, fish sauce, l..."
3,12,"[sugar, vanilla extract, corn starch, coffee g..."
4,13,"[pie crust, bourbon whiskey, powdered sugar, c..."


In [14]:
train_data['cuisine'].value_counts()

italian         7838
mexican         6438
southern_us     4320
indian          3003
chinese         2673
french          2646
cajun_creole    1546
thai            1539
japanese        1423
greek           1175
spanish          989
korean           830
vietnamese       825
moroccan         821
british          804
filipino         755
irish            667
jamaican         526
russian          489
brazilian        467
Name: cuisine, dtype: int64

In [19]:
#keep count of no. ingredients in train 
train_ingredients_count={}
for i in range(len(train_data)):
    for j in train_data['ingredients'][i]:
        if j in train_ingredients_count.keys():
            train_ingredients_count[j]+=1
        else:
            train_ingredients_count[j]=1

In [20]:
test_ingredients_count={}
for i in range(len(test_data)):
    for j in test_data['ingredients'][i]:
        if j in test_ingredients_count.keys():
            test_ingredients_count[j]+=1
        else:
            test_ingredients_count[j]=1

In [21]:
#print(len(train_ingredients_count))
#print(len(test_ingredients_count))

6215
4121


In [22]:
# count no. ingredients in test, not in train
ingredients_missing_train=[]
for i in test_ingredients_count.keys():
    if i not in train_ingredients_count.keys():
        ingredients_missing_train.append(i)
print(len(ingredients_missing_train)) 

399


In [23]:
for i in ingredients_missing_train:
    train_ingredients_count[i]=0
print(len(train_ingredients_count))  

6614


In [24]:
ingredients_missing=[]
for i in train_ingredients_count.keys():
    if i not in test_ingredients_count.keys():
        ingredients_missing.append(i)
print(len(ingredients_missing))   

2493


In [25]:
for i in ingredients_missing:
    test_ingredients_count[i]=0
print(len(test_ingredients_count))    

6614


In [26]:
for i in train_ingredients_count.keys():
    train_data[i]=np.zeros(len(train_data))

In [27]:
for i in test_ingredients_count.keys():
    test_data[i]=np.zeros(len(test_data))

In [28]:
print(train_data.shape)
print(test_data.shape)

(39774, 6617)
(9944, 6616)


In [29]:
for i in range(len(train_data)):
    for j in train_data['ingredients'][i]:
        train_data[j].iloc[i]=1

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [30]:
#train_data.head()

In [31]:
for i in range(len(test_data)):
    for j in test_data['ingredients'][i]:
        test_data[j].iloc[i]=1

In [32]:
#test_data.head()

In [33]:
test_data=test_data[train_data.drop('cuisine',axis=1).columns]

In [34]:
from sklearn.model_selection import train_test_split
X=train_data.drop(['id','ingredients','cuisine'],axis=1)
y=train_data['cuisine']

In [35]:
X_train,X_val,y_train,y_val=train_test_split(X,y,random_state=42)

In [36]:
print(X_train.shape,y_train.shape)
print(X_val.shape,y_val.shape)

(29830, 6614) (29830,)
(9944, 6614) (9944,)


## Fit Linear SVC

In [39]:
from sklearn.svm import LinearSVC, SVC
from sklearn.metrics import f1_score

from sklearn.model_selection import train_test_split, GridSearchCV

In [40]:

param_grid = {'C': [0.001,0.01, 0.1, 1, 10, 50, 100, 500, 1000,5000],  
              'penalty': ['l1','l2'],
             'loss': ['hinge','squared hinge']} 

grid = GridSearchCV(LinearSVC(), param_grid, refit = True, verbose = 3, n_jobs=-1)

In [41]:
grid.fit(X_train,y_train) 

Fitting 5 folds for each of 40 candidates, totalling 200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:  9.6min
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed: 17.2min finished
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


GridSearchCV(estimator=LinearSVC(), n_jobs=-1,
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 50, 100, 500, 1000,
                               5000],
                         'loss': ['hinge', 'squared hinge'],
                         'penalty': ['l1', 'l2']},
             verbose=3)

In [42]:
grid.best_params_

{'C': 1, 'loss': 'hinge', 'penalty': 'l2'}

In [ ]:
# kaggle score: 0.768